In [1]:
!pip install -U -q "tf-models-official"

In [2]:
import os
import cv2
import zipfile
import numpy as np
import tensorflow as tf
import moviepy.editor as mp
from collections import defaultdict
from official.projects.movinet.modeling import movinet, movinet_model

2025-04-19 03:37:58.501848: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745033878.524717     109 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745033878.531545     109 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745033878.550014     109 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745033878.550038     109 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745033878.550040     109 computation_placer.cc:177] computation placer alr

In [3]:
def preprocess_video(video_path, num_frames=16, resolution=224):
    print(f"Attempting to open video: {video_path}")  
    cap = cv2.VideoCapture(video_path, cv2.CAP_FFMPEG, params=[])
    if not cap.isOpened():
        raise ValueError(f"Could not open video file: {video_path}. Ensure FFmpeg is installed and file is valid.")
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_step = max(1, total_frames // num_frames)
    
    for i in range(0, total_frames, frame_step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (resolution, resolution))
        frame = frame / 255.0  
        frames.append(frame)
    
    cap.release()
    while len(frames) < num_frames:
        frames.append(np.zeros((resolution, resolution, 3)))
    return np.array(frames[:num_frames])

def predict_events(model, video_data, class_names, threshold=0.5):
    event_timestamps = []
    print(f"Predicting events from preprocessed data with shape: {video_data.shape}")
    video_data = np.expand_dims(video_data, axis=0)
    predictions = model.predict(video_data, verbose=0)
    print(f"Raw predictions: {predictions}")
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS) if 'video_path' in globals() else 30.0  
    total_frames = video_data.shape[0] * (max(1, int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FRAME_COUNT)) // video_data.shape[1]))
    for i in range(0, total_frames, max(1, total_frames // video_data.shape[1])):
        pred = predictions[0]
        for idx, confidence in enumerate(pred):
            if confidence > threshold:
                timestamp = (i / fps) if i < total_frames else (total_frames / fps) / 2  
                event_timestamps.append((timestamp, class_names[idx], confidence))
    return event_timestamps

def preprocess_video_segment(frame, num_frames, resolution):
    frames = [frame]
    while len(frames) < num_frames:
        frames.append(np.zeros((resolution, resolution, 3)))
    return np.array(frames[:num_frames]) / 255.0

def create_highlight_reel(video_path, event_timestamps, output_path="/kaggle/working/highlight_reel_2.mp4", window=5):
    if not event_timestamps:
        raise ValueError("No events detected to create a highlight reel.")
    clip = mp.VideoFileClip(video_path)
    print(f"Video duration: {clip.duration} seconds")
    print(f"Creating highlight reel with {len(event_timestamps)} events: {event_timestamps}")  
    
    timestamp_groups = defaultdict(list)
    for timestamp, event, confidence in event_timestamps:
        timestamp_groups[timestamp].append((event, confidence))
    
    clips = []
    for timestamp, events in timestamp_groups.items():
        try:
            start_time = max(0, timestamp - window)
            end_time = min(clip.duration, timestamp + window)
            if start_time >= end_time:
                print(f"Skipping invalid timestamp {timestamp:.2f}s: start_time ({start_time:.2f}s) >= end_time ({end_time:.2f}s)")
                continue
            event_clip = clip.subclip(start_time, end_time)
            if event_clip is None or event_clip.duration <= 0:
                print(f"Invalid clip at {timestamp:.2f}s: duration {event_clip.duration if event_clip else 'None'}")
                continue
            clips.append(event_clip)
            print(f"Added clip from {start_time:.2f}s to {end_time:.2f}s, duration {event_clip.duration:.2f}s for events {events}")
        except Exception as e:
            print(f"Failed to add clip at timestamp {timestamp:.2f}s: {e}")
    
    if not clips:
        raise ValueError("No valid clips were created for the highlight reel.")
    final_clip = mp.concatenate_videoclips(clips)
    final_clip.write_videofile(output_path, codec="libx264")
    return output_path

In [4]:
def test_model(weights_path, video_path, class_names):
    
    batch_size = 1
    num_frames = 16
    resolution = 224
    backbone = movinet.Movinet(model_id='a3')
    backbone.build([batch_size, num_frames, resolution, resolution, 3])
    model = movinet_model.MovinetClassifier(
        backbone=backbone,
        num_classes=len(class_names),
        dropout_rate=0.7
    )
    model.build([batch_size, num_frames, resolution, resolution, 3])
    model.load_weights(weights_path)
    
    
    video_data = preprocess_video(video_path, num_frames, resolution)
    event_timestamps = predict_events(model, video_data, class_names)
    
    
    if event_timestamps:
        print("Detected events:")
        for timestamp, event, confidence in event_timestamps:
            print(f"Event: {event}, Timestamp: {timestamp:.2f}s, Confidence: {confidence:.4f}")
    else:
        print("No events detected with sufficient confidence.")
    
    if event_timestamps:
        output_path = create_highlight_reel(video_path, event_timestamps)
        with zipfile.ZipFile("/kaggle/working/output_reel.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
            zipf.write(output_path, os.path.basename(output_path))
        print(f"Highlight reel generated: {output_path}. Download 'output.zip' from the Output tab.")
    else:
        print("No highlight reel generated due to no detected events.")

In [5]:
class_names = ['Corner', 'Direct free-kick', 'Foul', 'Goal', 'Indirect free-kick', 
               'Kick-off', 'Offside', 'Shots off target', 'Shots on target', 
               'Substitution', 'Throw-in', 'Yellow card']

weights_path = "/kaggle/input/movinet-model-latest/keras/default/1/movinet_classifier_weights_epoch_10.weights.h5"
video_path = "/kaggle/input/test-match-dataset/2_720p.mkv"
# video_path = "/kaggle/input/test-match-dataset/1_720p.mkv"

test_model(weights_path, video_path, class_names)

I0000 00:00:1745033891.865591     109 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Attempting to open video: /kaggle/input/test-match-dataset/2_720p.mkv
Predicting events from preprocessed data with shape: (16, 224, 224, 3)


I0000 00:00:1745033923.875854     170 service.cc:152] XLA service 0x7f0bd4002450 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745033923.875908     170 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745033923.898941     170 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745033924.488524     170 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Raw predictions: [[ 0.31843692 -1.0027473  -1.9367456  -1.8451654  -0.67007846  1.1106658
  -3.4565277  -2.8935723  -3.9636736  -4.601125   -0.52038383 -1.0191002 ]]
Detected events:
Event: Kick-off, Timestamp: 0.00s, Confidence: 1.1107
Event: Kick-off, Timestamp: 10.96s, Confidence: 1.1107
Event: Kick-off, Timestamp: 21.92s, Confidence: 1.1107
Event: Kick-off, Timestamp: 32.88s, Confidence: 1.1107
Event: Kick-off, Timestamp: 43.84s, Confidence: 1.1107
Event: Kick-off, Timestamp: 54.80s, Confidence: 1.1107
Event: Kick-off, Timestamp: 65.76s, Confidence: 1.1107
Event: Kick-off, Timestamp: 76.72s, Confidence: 1.1107
Event: Kick-off, Timestamp: 87.68s, Confidence: 1.1107
Event: Kick-off, Timestamp: 98.64s, Confidence: 1.1107
Event: Kick-off, Timestamp: 109.60s, Confidence: 1.1107
Event: Kick-off, Timestamp: 120.56s, Confidence: 1.1107
Event: Kick-off, Timestamp: 131.52s, Confidence: 1.1107
Event: Kick-off, Timestamp: 142.48s, Confidence: 1.1107
Event: Kick-off, Timestamp: 153.44s, Confide

MoviePy - Done.
Moviepy - Writing video /kaggle/working/highlight_reel_2.mp4



Moviepy - Done !
Moviepy - video ready /kaggle/working/highlight_reel_2.mp4
Highlight reel generated: /kaggle/working/highlight_reel_2.mp4. Download 'output.zip' from the Output tab.
